# import dependancies

In [40]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd

# database connection

In [41]:
conn = psycopg2.connect(database="postgres", user="postgres", password="postgres", host="localhost", port = "5432")
cur = conn.cursor()

## import data function

In [42]:
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
  
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

# load data

In [43]:
df = pd.read_csv('https://s3.amazonaws.com/coderbyteprojectattachments/simcel-6pk70-1jk5iqdp-train_v9rqX0R.csv')

# process data

In [44]:
item = df[['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Type']]
item = item.sort_values(by=['Item_Identifier', 'Item_Weight'], ascending=True)
item.ffill(inplace=True)
# item['Item_Fat_Content'].unique()
# item['Item_Type'].unique()
item['Item_Fat_Content'] = item['Item_Fat_Content'].replace('LF', 'Low Fat').replace('low fat', 'Low Fat').replace('reg', 'Regular')
item.drop_duplicates(inplace=True)

In [45]:
outlet = df[['Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']]
outlet.drop_duplicates(inplace=True)

C:\Users\anhpc\AppData\Local\Temp\ipykernel_536\2712254571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlet.drop_duplicates(inplace=True)


In [46]:
sale_report = df[['Item_Identifier', 'Outlet_Identifier', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales']]

# create tables and import data to database

## item table

In [47]:
sql = """
CREATE TABLE item (
    item_identifier CHAR(5) PRIMARY KEY,
    item_weight REAL,
    item_fat_content VARCHAR(10),
    item_type VARCHAR(30)
);
"""
cur.execute(sql)

In [48]:
execute_values(conn, item, 'item')

execute_values() done


## store table

In [49]:
sql = """
CREATE TABLE outlet (
    outlet_identifier CHAR(6) PRIMARY KEY,
    outlet_establishment_year SMALLINT,
    outlet_size VARCHAR(10),
    outlet_location_Type CHAR(6),
    outlet_type VARCHAR(20)
);
"""
cur.execute(sql)

In [50]:
execute_values(conn, outlet, 'outlet')

execute_values() done


## sale_report table

In [51]:
sql = """
CREATE TABLE sale_report (
    id SERIAL PRIMARY KEY,
    item_identifier CHAR(5) REFERENCES item(Item_Identifier),
    outlet_identifier CHAR(6) REFERENCES outlet(Outlet_Identifier),
    item_visibility REAL,
    item_mrp REAL,
    item_outlet_sales REAL
);
"""
cur.execute(sql)

In [52]:
sale_report['id'] = sale_report.index
execute_values(conn, sale_report, 'sale_report')

C:\Users\anhpc\AppData\Local\Temp\ipykernel_536\1767192921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sale_report['id'] = sale_report.index


execute_values() done
